In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from torch.utils.data import DataLoader
from dataset import dataset_hparams, MaestroDataset
from model import model_hparams, Model

In [9]:
maestro_dataset = MaestroDataset(dataset_hparams)
print(maestro_dataset)

train_loader = DataLoader(maestro_dataset, batch_size=16)
print(train_loader)

In [10]:
maestro_dataset[0]

array([  8, 327,   0, 336,   1, 339,   0, 110, 197,   0, 112, 209,   0,
       113, 212,   8, 337,   1, 340,   0, 112, 200,   1, 325,   8, 328,
         0, 114, 204,   0, 116, 207,   0, 112, 202,  10, 330,   0, 113,
       200,   7, 332,   2, 328,   1, 115, 209,   0, 110, 199,   0, 110,
       206,   0, 335,   8, 327,   1, 108, 197,   6, 337,   1, 334,   0,
       110, 207,   0, 111, 211,   1, 325,   0, 110, 195,   2, 335,   6,
       339,   2, 110, 209,   0, 108, 202,   0, 108, 206,   3, 323,   3,
       334,   2, 330,   0, 337,   1, 111, 204,   0, 115, 212,   0, 111,
       207,   4, 335,   2, 340,   2, 115, 206,   0, 116, 211,   0, 113,
       202,   0, 332,   1, 339,   1, 334,   5, 119, 209,   0, 115, 200,
         0, 113, 204,   0, 330,   2, 328,   1, 332,   2, 337,   2, 118,
       207,   0, 113, 199,   0, 115, 202,   2, 330,   0, 335,   3, 327,
         2, 117, 206,   0, 115, 200,   0, 113, 197,   7, 334,   0, 328,
         0, 325,   0, 116, 204,   0, 114, 195,   0, 114, 199,   

In [11]:
def tokens_to_event_list(tokens, hp):
    current_time = 0
    current_velocity = 0
    event_list = []
    for token in tokens:
        # interval
        if token < hp.offsets.velocity:
            current_time += token / hp.dims.interval * hp.max_note_duration
            
        # velocity
        elif token < hp.offsets.note_on:
            current_velocity = (token - hp.offsets.velocity) / hp.dims.velocity * 128
            
        # note_on
        elif token < hp.offsets.note_off:
            event = {'time': current_time,
                     'type': 'note_on',
                     'note': token - hp.offsets.note_on,
                     'velocity': int(current_velocity)}
            event_list.append(event)
                     
        # note_off
        elif token < hp.offsets.pedal_on:
            event = {'time': current_time,
                     'type': 'note_off',
                     'note': token - hp.offsets.note_off}
            event_list.append(event)
            
        # pedal_on
        elif token < hp.offsets.pedal_off:
            event = {'time': current_time,
                     'type': 'pedal_on'}
            event_list.append(event)
            
        # pedal_off
        else:
            event = {'time': current_time,
                     'type': 'pedal_off'}
            event_list.append(event)
    
    return event_list
    

In [12]:
def save_event_list_to_midi_file(event_list, midi_file):
    from midiutil import MIDIFile
    '''
    Reference : https://midiutil.readthedocs.io/en/1.2.1/index.html
    '''
    track    = 0
    channel  = 0
    tempo    = 60  # In BPM
    MyMIDI = MIDIFile(1) # One track, defaults to format 1 (tempo track
                         # automatically created)
    MyMIDI.addTempo(track, 0, tempo)
    
    notes = [None for _ in range(128)]
    for event in event_list:
        if event['type'] == 'note_on':
            notes[event['note']] = event
            
        elif event['type'] == 'note_off' and notes[event['note']] is not None:
            pitch = event['note']
            start_time = notes[event['note']]['time']
            duration = max(event['time'] - start_time, 0)
            velocity = notes[event['note']]['velocity']
            MyMIDI.addNote(track, channel, pitch, start_time, duration, velocity)
            notes[event['note']] = None
            
        elif event['type'] == 'pedal_on':
            MyMIDI.addControllerEvent(track, channel, event['time'], 64, 1)
            
        elif event['type'] == 'pedal_off':
            MyMIDI.addControllerEvent(track, channel, event['time'], 64, 0)
            
    with open(midi_file, "wb") as output_file:
        MyMIDI.writeFile(output_file)
    

In [13]:
event_list = tokens_to_event_list(maestro_dataset[0], dataset_hparams)
for event in event_list:
    print(event)

{'time': 0.0, 'type': 'note_off', 'note': 70}
{'time': 0.0, 'type': 'note_off', 'note': 77}
{'time': 0.0, 'type': 'note_off', 'note': 62}
{'time': 0.02, 'type': 'note_off', 'note': 74}
{'time': 0.08, 'type': 'note_off', 'note': 58}
{'time': 0.38, 'type': 'note_on', 'note': 79, 'velocity': 72}
{'time': 0.46, 'type': 'note_off', 'note': 79}
{'time': 0.46, 'type': 'note_on', 'note': 77, 'velocity': 84}
{'time': 0.56, 'type': 'note_on', 'note': 75, 'velocity': 76}
{'time': 0.56, 'type': 'note_off', 'note': 77}
{'time': 0.64, 'type': 'note_on', 'note': 74, 'velocity': 76}
{'time': 0.64, 'type': 'note_off', 'note': 75}
{'time': 0.7, 'type': 'note_off', 'note': 74}
{'time': 0.72, 'type': 'note_on', 'note': 72, 'velocity': 72}
{'time': 0.74, 'type': 'pedal_off'}
{'time': 0.78, 'type': 'note_off', 'note': 72}
{'time': 0.78, 'type': 'note_on', 'note': 70, 'velocity': 76}
{'time': 0.86, 'type': 'note_off', 'note': 70}
{'time': 0.86, 'type': 'note_on', 'note': 69, 'velocity': 80}
{'time': 0.86, 't

In [14]:
save_event_list_to_midi_file(event_list, 'sample.mid')